# Libraries

In [275]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 
from geopy.distance import geodesic
from datetime import datetime, timedelta, time, date
import matplotlib.pyplot as plt
# Part 2 (IFAM model)

In [276]:
# group_number gn 
gn = '18'

# Inputs

In [277]:
# Define the inputs

# L: set of flights
flights_df = pd.read_excel('Assignment_Data/Part_2/Group_'+gn+'_P2.xlsx', sheet_name='Flight')
#change data type of colum 'Flight no.' ro string
flights_df['Flight Number'] = flights_df['Flight Number'].astype(str)
flights_list = flights_df['Flight Number'].to_list() 

# P: set of passenger itineraries
paths = pd.read_excel('Assignment_Data/Part_2/Group_'+gn+'_P2.xlsx', sheet_name='Itinerary').set_index('Itin No.')

# P_p: set of passenger itineraries with Recapture Rate from itinerary p
recapture_p = pd.read_excel('Assignment_Data/Part_2/Group_'+gn+'_P2.xlsx', sheet_name='Recapture Rate').set_index(['From Itinerary','To Itinerary'])
recapture_p.rename(columns={'From Itinerary': 'p', 'To Itinerary': 'r'}, inplace=True)

# K: set of aircraft types
aircraft_df = pd.read_excel('Assignment_Data/Part_2/Group_'+gn+'_P2.xlsx', sheet_name='Aircraft')
aircraft_df.rename(columns={'Type': 'AC Type'}, inplace=True)
aircraft_df.set_index('AC Type', inplace=True)
aircraft = aircraft_df.to_dict(orient='index')
ac_list = list(aircraft.keys())

# make a dictionary with the itinerary as the key and the rest as a sub-dictionary
paths['Leg 1'] = paths['Leg 1'].astype(str)
paths['Leg 2'] = paths['Leg 2'].astype(str)

paths = paths.to_dict(orient='index')
path_list = list(paths.keys())

# flights = flights_df.to_dict(orient='index')
recapture_p = recapture_p.to_dict(orient='index')
# Drop the inner dicts and just keep the values of the inner dict
for key in recapture_p:
    recapture_p[key] = recapture_p[key]['Recapture Rate']

# For all paths if 'Leg 1' and 'Leg 2' are numbers then create a list with both legs else, drop the keys from the list, and create a new key called 'Legs'
# else just change the name of the key 'Leg 1' to 'Legs'
for key in path_list:
    legs = []
    if paths[key]['Leg 1'] != '0' and paths[key]['Leg 2'] != '0':
        legs.append(paths[key]['Leg 1'])
        legs.append(paths[key]['Leg 2'])
        paths[key]['Legs'] = legs
    elif paths[key]['Leg 1'] != '0':
        legs.append(paths[key]['Leg 1'])
        paths[key]['Legs'] = legs
    del paths[key]['Leg 1']
    del paths[key]['Leg 2']

# Define path 999 with a fare of 0 and a demand of 0 
paths[999] = {'Legs': [], 'Demand': 0, 'Fare': 0}

for k in aircraft:
    aircraft[k]['TAT'] = timedelta(minutes=aircraft[k]['TAT'])

flights = flights_df.merge(aircraft_df.reset_index()[['AC Type']], how='cross')

In [278]:
# Get the cost from the column named after the AC Type
flights['Cost'] = flights.apply(lambda row: row[row['AC Type']], axis=1)
flights.drop(columns=ac_list, inplace=True)

# Get the capacity from the dictionary
flights['Capacity'] = flights.apply(lambda row: aircraft[row['AC Type']]['Seats'], axis=1)

In [279]:
# Read the data distance file
distance_info = pd.read_csv('Assignment_Data/Group_4_Distances.csv')
distance_info.rename(columns={'Unnamed: 0': 'Origin'}, inplace=True)

# Create a dictionary with the distance info
distance = {}
for index, row in distance_info.iterrows():
    distance[row['Origin']] = row.to_dict()

# Remove the origin column from the distance
for i in distance:
    distance[i].pop('Origin', None)

In [280]:
# List of unique airports from Origin and Destination columns
airports = list(set(flights['ORG'].unique()).union(set(flights['DEST'].unique())))

In [281]:
# Misc date just for adding the TAT
misc_date = date(1,1,1)

# Drop rows with distance > range
flights['Arrival'] = flights.apply(lambda row: (datetime.combine(misc_date,row['Arrival']) + aircraft[row['AC Type']]['TAT']).time(), axis=1)
flights['Overnight'] = flights.apply(lambda row: row['Arrival'] < row['Departure'], axis=1)
# Make flights dictionary with main keys: AC Type, with a sub dictionary of flight numbers and each with flight details
flights_dict = {}
for i in flights_list:
    flights_dict[i] = flights[flights['Flight Number'] == i].set_index('AC Type').to_dict(orient='index')

flights = flights_dict

In [282]:
# Create an empty list to store the data
data = []
# Iterate over airports, aircraft types, and flights
for l in flights:
    for k in flights[l]:
            for n in airports:
                if flights[l][k]['ORG'] == n:
                    data.append([k, n, l, flights[l][k]['Departure'], 'Departure'])
                if flights[l][k]['DEST'] == n:
                    data.append([k, n, l, flights[l][k]['Arrival'], 'Arrival'])

# Create a dataframe with the data
events = pd.DataFrame(data, columns=['AC Type','Airport', 'Flight N', 'Time', 'D_A'])

# Add the TAT to the arrival times

events.sort_values(by=['AC Type', 'Airport', 'Time'], inplace=True)

# Reset the numbering of the events
events.reset_index(drop=True, inplace=True)

In [283]:
# For each airport and aircraft type i need to create a loop of ground arcs, each starting from the last event and ending at the next event, if there is no more events then the last ground arc is the overnight arc and it ends at the first event of the next day (first event of the next day is the first event of the same airport and aircraft type)

ground_arcs = pd.DataFrame(columns=['AC Type', 'Airport', 'Start Time', 'End Time'])

for k in ac_list:
    for n in airports:
        df = events[(events['AC Type'] == k) & (events['Airport'] == n)].sort_values(by=['Time'])
        for i in range(len(df)):
            if i == 0:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)
            else:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[i-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)

ground_arcs.sort_values(by=['AC Type', 'Airport'], inplace=True)
# Drop rows if start time and end time are the same
ground_arcs = ground_arcs[~ground_arcs.apply(lambda row: (row['Start Time'] == row['End Time']), axis=1)]
ground_arcs['Arc ID'] = ground_arcs.groupby(['AC Type']).cumcount()
ground_arcs['Overnight'] = ground_arcs.apply(lambda row: (row['End Time'] < row['Start Time']), axis=1)
ground_arcs.sort_values(by='Arc ID', inplace=True)

In [284]:
# Create a nodes_df
nodes_df = ground_arcs[['AC Type', 'Airport', 'Start Time']].rename(columns={'Start Time': 'Time'})

# Add a count number for each row group by AC Type and Airport
nodes_df['Node ID'] = nodes_df.groupby(['AC Type']).cumcount()

# make a dictionary with the ac type as main key and the airport as secondary key with the node as tertiary key and the time as value
nodes = {}
for k in ac_list:
    nodes[k] = {}
    for n in airports:
        nodes[k][n] = {}
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i] = {'Time': nodes_df[(nodes_df['AC Type'] == k) & 
                                               (nodes_df['Airport'] == n) & 
                                               (nodes_df['Node ID'] == i)]['Time'].values[0]}
            

In [ ]:
# Add node id to the events
events.merge(nodes_df, how='left', on=['AC Type', 'Airport', 'Time']).sort_values(by=['AC Type', 'Airport'])

# Add a dictionary call departures and another one arrivals to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['Departures'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Departure')]['Flight N'])
            nodes[k][n][i]['Arrivals'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Arrival')]['Flight N'])


In [ ]:
# n+: ground arcs originating at any node n (start time)
# n-: ground arcs ending at any node n (end time)
n_plus = ground_arcs[['Airport', 'AC Type', 'Start Time', 'Arc ID']].rename(columns={'Start Time': 'Time'})
n_minus = ground_arcs[['Airport', 'AC Type', 'End Time', 'Arc ID']].rename(columns={'End Time': 'Time'})

# Add a dictionary call n+ and another one n- to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['n+'] = list(n_plus[(n_plus['AC Type'] == k) & (n_plus['Airport'] == n) & (n_plus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])
            nodes[k][n][i]['n-'] = list(n_minus[(n_minus['AC Type'] == k) & (n_minus['Airport'] == n) & (n_minus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])

In [ ]:
overnight_arcs = ground_arcs[ground_arcs['Overnight'] == True][['AC Type', 'Airport', 'Arc ID']]
overnight_flights = []
for l in flights:
    for k in flights[l]:
        if flights[l][k]['Overnight']:
            overnight_flights.append([k, l])

overnight_flights = pd.DataFrame(overnight_flights, columns=['AC Type', 'Flight no.'])

In [ ]:
# s_ip: binary variable indicating whether flight i is in itinerary p
s_ip = {}
for i in flights_list:
    for p in paths:
        s_ip[i,999] = 0
        if i in paths[p]['Legs']:
            s_ip[i,p] = 1
        else:
            s_ip[i,p] = 0

# Q_i: unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
Q_i = {}
for i in flights_list:
    Q_i[i] = 0
    for p in paths:
        Q_i[i] += s_ip[i,p] * paths[p]['Demand']

In [ ]:
path_list = list(paths.keys())
flight_list = list(flights.keys())

In [ ]:
# Add entries to P_p for path 0 with a Recapture Rate of 1
for p in paths:
    recapture_p[p,999] = 1
    recapture_p[999,p] = 0

In [ ]:
plotiing = False
if plotiing:
    import random
    import plotly.graph_objects as go

    for k in ac_list:
        df = events[events['AC Type'] == k].sort_values(by=['Time'])
        fig = go.Figure()
        
        # Set marker color based on 'D_A' column
        marker_color = ['red' if d_a == 'Departure' else 'blue' for d_a in df['D_A']]
        
        # Add ground arcs
        ground_arcs_k = ground_arcs[ground_arcs['AC Type'] == k]
        for i, row in ground_arcs_k.iterrows():
            # Generate a random color
            random_color = '#' + ''.join(random.choices('0123456789ABCDEF', k=6))
            fig.add_shape(
                type="line",
                x0=row['Start Time'],
                y0=row['Airport'],
                x1=row['End Time'],
                y1=row['Airport'],
                line=dict(color=random_color, width=2)
            )
        
        fig.add_trace(go.Scatter(
            x=df['Time'], 
            y=df['Airport'], 
            mode='markers+text',
            marker=dict(color=marker_color),
            hovertemplate= '<b>Flight no.</b>: ' + df['Flight N'] + '<br>' ))
        
        
        fig.update_layout(
            title="Ground Arcs for AC Type " + k,
            xaxis_title="Time",
            yaxis_title="Airport",
        )
        fig.show()


In [ ]:
# for each path p, key-value pair in initial pairs with key p and value 999 in a set
initial_pairs = {}
for p in path_list:
    initial_pairs[p] = set()
    initial_pairs[p].add(999)

In [ ]:
# Notation

L = flights
P = paths
P0 = initial_pairs
K = ac_list
G = nodes
N = airports
Q = Q_i
R = recapture_p
Gr = ground_arcs


# Iterative model

In [ ]:
def PMF_n_iters(
        n_iters, pairs, # required inputs
        pi = {}, sigma= {}, P=P, L=L, 
        R=R, iter=0): # optional inputs
    
    if iter == 0:
        print('Base model: all itineraries to 999')
        C = {k: set(v) for k, v in pairs.items()} # current pairs

    if iter > 0:
        print('Iteration number: ', iter)
        C = {k: set(v) for k, v in pairs.items()} # current pairs
        tpr_prime = {}
        # tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
        for p,r in R.keys():
            t_prime_pr = ((P[p]['Fare'] - sum(pi[i] for i in P[p]['Legs'])) -
                            (R[(p,r)]) *
                            (P[r]['Fare'] - sum(pi[j] for j in P[r]['Legs'])) -
                            (sigma[p]))
            if t_prime_pr < -0.00001:
                tpr_prime[p,r] = t_prime_pr
                print(str(p)+'->'+str(r)+': ', t_prime_pr)
                
        pairs = list(tpr_prime.keys())
        
        for n_p in pairs:
            C[n_p[0]].add(n_p[1])


    if len(pairs) == 0:
        print('No new pairs, optimal solution found in previous iteration')
        return C, pi, sigma
    
    if len(pairs) > 0:
        print('New pairs: ', pairs)
        # print('Current pairs: ', current_pairs)
        # Define the model
        m_n = gp.Model('IFAM')

        # Decision variables from FAM
        # f[i,k] [RELAXED] binary 1 if flight arc i is assigned to aircraft type k, 0 otherwise
        f = {}
        # y_ak = [RELAXED] integer number of aircraft of type k on the ground arc a
        y = {}

        for i in L:
            for k in K:
                f[i, k] = m_n.addVar(vtype=GRB.CONTINUOUS, name='f_' + str(i) + '_' + str(k))

        for k in K:
            for a in list(ground_arcs[(ground_arcs['AC Type'] == k)]['Arc ID']):
                y[a, k] = m_n.addVar(vtype=GRB.CONTINUOUS,lb=0, name='y_' + str(a) + '_' + str(k))

        # Decision variables from PMF
        # t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
        t = {}

        for p in C:
            for r in C[p]:
                t[p,r] = m_n.addVar(vtype=GRB.CONTINUOUS,lb=0,name='t_'+str(p)+'_'+str(r))
        m_n.update()

        # Objective function part from the FAM
        of = gp.quicksum(
            L[i][k]['Cost'] * 
            f[i,k] 
            for i in L for k in K)

        # Objective function part from the PMF
        of +=  gp.quicksum((P[p]['Fare'] - R[(p,r)] * P[r]['Fare']) * t[p,r] 
                        for p in C for r in C[p])

        # Define the objective function
        m_n.setObjective(of, GRB.MINIMIZE)

        # Define the constraints
        # Constraint 1 [FAM]: 
        # Each flight is assigned to exactly one aircraft type
        for i in L:
            m_n.addConstr((gp.quicksum(f[i,k] for k in K) == 1), name='one_ac')

        # Constraint 2 [FAM]: 
        # The number of AC arriving = AC departing, for each type at each node
        # y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
        for k in K:
            for n in airports:
                for i in nodes[k][n]:
                    m_n.addConstr((y[nodes[k][n][i]['n+'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Departures']) == 
                                y[nodes[k][n][i]['n-'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Arrivals']) ),
                                name='balance_' + str(i) + '_' + str(k) + '_' + str(n))

        # Constraint 3 [FAM]: 
        # The number of overnight arcs + the number of overnight flights = the number of aircraft of each type 
        # using overnight_arcs and overnight_flights
        # sum(y_a,k) + sum(f_i,k) = number of aircraft of type k
        for k in K:
            m_n.addConstr((gp.quicksum(y[a, k] for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID'])) + 
                        gp.quicksum(f[i, k] for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.'])) == 
                        aircraft[k]['Units']), name='overnight_' + str(k))

        # Constraint 4 [MIXED]: 
        # removed (from flight i in path p) - recaptured (for flight i in path p) ≥ demand spillage (for flight i) - capacity (for flight i) assinged to aircraft type k
        # sum seats_k * f_ik -sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
        m_n.addConstrs((
            gp.quicksum(s_ip[i,p] * t[p,r] for p in C for r in C[p]) - 
            gp.quicksum(s_ip[i,r] * R[(p,r)] * t[p,r] for p in C for r in C[p]) >= 
            Q[i] - gp.quicksum(aircraft[k]['Seats'] * f[i,k] for k in K) for i in L), name='π')

        # Constraint 5 [PMF]: sum t_pr <= Dp for all p
        for p in C:
            m_n.addConstr((
                gp.quicksum(t[p,r] for r in C[p]) <= 
                P[p]['Demand']), name='σ[' + str(p) + ']')

        # Update the model
        m_n.update()
        # Optimize the model but dont print the output
        m_n.setParam('OutputFlag', 0)
        m_n.optimize()
        print('Objective value: %0.0f' % (m_n.objVal))


        # Save dual variables in a dictionary
        pi_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'π':
                flight_num_pi = c.ConstrName[2:-1]
                pi_new[flight_num_pi] = c.Pi

        sigma_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'σ':
                path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
                sigma_new[path_num_sigma] = c.Pi

        if iter == 0:
            print ('End of base model iteration\n')
        else:
            print('End of iteration number: ', iter, '\n')
        
        iter += 1

        if iter == n_iters:
            print('Max number of iterations reached')
            return C, pi_new, sigma_new
        else:
            return  PMF_n_iters(
                    n_iters,
                    C,
                    sigma = sigma_new, 
                    pi= pi_new,
                    iter=iter)

In [273]:
final_pairs, final_pi, final_sigma = PMF_n_iters(21, initial_pairs)

Base model: all itineraries to 999
New pairs:  {0: {999}, 1: {999}, 2: {999}, 3: {999}, 4: {999}, 5: {999}, 6: {999}, 7: {999}, 8: {999}, 9: {999}, 10: {999}, 11: {999}, 12: {999}, 13: {999}, 14: {999}, 15: {999}, 16: {999}, 17: {999}, 18: {999}, 19: {999}, 20: {999}, 21: {999}, 22: {999}, 23: {999}, 24: {999}, 25: {999}, 26: {999}, 27: {999}, 28: {999}, 29: {999}, 30: {999}, 31: {999}, 32: {999}, 33: {999}, 34: {999}, 35: {999}, 36: {999}, 37: {999}, 38: {999}, 39: {999}, 40: {999}, 41: {999}, 42: {999}, 43: {999}, 44: {999}, 45: {999}, 46: {999}, 47: {999}, 48: {999}, 49: {999}, 50: {999}, 51: {999}, 52: {999}, 53: {999}, 54: {999}, 55: {999}, 56: {999}, 57: {999}, 58: {999}, 59: {999}, 60: {999}, 61: {999}, 62: {999}, 63: {999}, 64: {999}, 65: {999}, 66: {999}, 67: {999}, 68: {999}, 69: {999}, 70: {999}, 71: {999}, 72: {999}, 73: {999}, 74: {999}, 75: {999}, 76: {999}, 77: {999}, 78: {999}, 79: {999}, 80: {999}, 81: {999}, 82: {999}, 83: {999}, 84: {999}, 85: {999}, 86: {999}, 87: {

Objective value: 5504833
End of base model iteration

Iteration number:  1
121->119:  -0.35
124->119:  -0.3900000000000001
133->75:  -0.7800000000000011
168->138:  -1.7000000000000002
316->276:  -0.5600000000000005
349->289:  -18.400000000000006
377->296:  -16.79
New pairs:  [(121, 119), (124, 119), (133, 75), (168, 138), (316, 276), (349, 289), (377, 296)]
Objective value: 5504288
End of iteration number:  1 

Iteration number:  2
No new pairs, optimal solution found in previous iteration


# Output testing

In [ ]:
for k in ac_list:
    for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID']):
        print(a, k, y[a, k].varName, y[a, k].x)
    for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.']):
        print(i, k, f[i,k].varName, f[i,k].x) 
    print(aircraft[k]['Units'])
    print('\n')
    

0 A330 y_0_A330 0.0
12 A330 y_12_A330 0.0
20 A330 y_20_A330 0.0
22 A330 y_22_A330 0.0
24 A330 y_24_A330 0.0
46 A330 y_46_A330 0.0
50 A330 y_50_A330 0.0
54 A330 y_54_A330 0.0
58 A330 y_58_A330 0.0
62 A330 y_62_A330 0.0
70 A330 y_70_A330 0.0
80 A330 y_80_A330 0.0
101 A330 y_101_A330 0.0
103 A330 y_103_A330 0.0
107 A330 y_107_A330 0.0
109 A330 y_109_A330 0.0
125 A330 y_125_A330 0.0
128 A330 y_128_A330 0.0
140 A330 y_140_A330 0.0
146 A330 y_146_A330 0.0
150 A330 y_150_A330 0.0
156 A330 y_156_A330 0.0
319 A330 y_319_A330 0.0
325 A330 y_325_A330 0.0
335 A330 y_335_A330 0.0
351 A330 y_351_A330 0.0
353 A330 y_353_A330 0.0
355 A330 y_355_A330 0.0
370 A330 y_370_A330 0.0
372 A330 y_372_A330 0.0
374 A330 y_374_A330 0.0
376 A330 y_376_A330 0.0
378 A330 y_378_A330 0.0
AR1132 A330 f_AR1132_A330 1.0
AR1133 A330 f_AR1133_A330 1.0
AR1160 A330 f_AR1160_A330 0.0
AR1161 A330 f_AR1161_A330 0.0
AR1254 A330 f_AR1254_A330 0.0
AR1257 A330 f_AR1257_A330 0.0
AR1302 A330 f_AR1302_A330 0.0
AR1303 A330 f_AR1303_A33

In [ ]:
for w in ac_list:
    print('Flights for AC Type ' + w)
    for i in flights_list:
        for k in ac_list:
            if f[i,k].x > 0 and k == w:
                print(i," | ", f[i,k].x," | ", ('Cost = %0.3f' % (flights[i][k]['Cost'] * f[i,k].x / 1000)))
    print('\n')

# Count the non null values of f
total_flights = 0
for v in m.getVars():
    if v.x != 0 and v.varName[0] == 'f':
        total_flights += v.x
print('Total flights: ' + str(total_flights))
print(sum(f[i,k].x for i in flights for k in ac_list))


Flights for AC Type A330
AR1132  |  1.0  |  Cost = 19.798
AR1133  |  1.0  |  Cost = 20.187
AR1322  |  1.0  |  Cost = 18.402
AR1323  |  1.0  |  Cost = 18.132
AR1376  |  1.0  |  Cost = 17.352
AR1377  |  1.0  |  Cost = 17.404
AR1920  |  1.0  |  Cost = 19.539
AR1921  |  1.0  |  Cost = 19.539


Flights for AC Type A340
AR1240  |  0.28750000000000003  |  Cost = 4.508
AR1241  |  0.2875  |  Cost = 4.588
AR1302  |  0.5  |  Cost = 11.822
AR1303  |  0.5  |  Cost = 11.412
AR1304  |  1.0  |  Cost = 21.734
AR1305  |  1.0  |  Cost = 22.132
AR1471  |  0.21249999999999997  |  Cost = 2.944
AR1472  |  0.21249999999999997  |  Cost = 2.857
AR1473  |  0.21249999999999997  |  Cost = 2.985
AR1474  |  0.21249999999999997  |  Cost = 2.902
AR1475  |  0.5  |  Cost = 7.024
AR1476  |  0.5  |  Cost = 6.827


Flights for AC Type B737
AR1140  |  1.0  |  Cost = 8.754
AR1402  |  1.0  |  Cost = 9.922
AR1403  |  1.0  |  Cost = 9.465
AR1406  |  1.0  |  Cost = 9.922
AR1412  |  1.0  |  Cost = 9.922
AR1413  |  1.0  |  Cost = 

In [ ]:
# Make a df with AC grounded at overnight on each airport
grounded_overnight = pd.DataFrame(columns=ac_list, index=airports)
overnight_arcs['count'] = overnight_arcs.apply(lambda row: y[row['Arc ID'], row['AC Type']].x, axis=1)

# Reorganize the df, make the columns the unique AC types and the rows the unique airports and the values the count
for k in ac_list:
    grounded_overnight[k] = overnight_arcs[overnight_arcs['AC Type'] == k].groupby(['Airport']).sum()['count']

# Fill the NaN values with 0
grounded_overnight.fillna(0, inplace=True)
grounded_overnight

,A330,A340,B737,B738
B,1.0,0.0,0.0,0.0
A,1.0,2.0,1.0,0.0
C,2.0,0.0,7.0,29.0


In [ ]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            # val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].varName) + ' (n+)'
            # val += y[nodes[k][n][i]['n+'][0], k].varName
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].varName) + ' (f+)'
                # val += f[w, k].varName
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].varName)  + ' (n-)'
            # val -= y[nodes[k][n][i]['n-'][0], k].varName
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].varName) + ' (f-)'
                # val -= f[w, k].varName
            # s += ' = ' + str(val)
            print(s)

146 | A330 | LGAV | 21:35:00 | y_146_A330 (n+) + f_AR1867_A330 (f+) = y_149_A330 (n-)
147 | A330 | LGAV | 07:40:00 | y_147_A330 (n+) = y_146_A330 (n-) + f_AR1864_A330 (f-)
148 | A330 | LGAV | 08:00:00 | y_148_A330 (n+) + f_AR1865_A330 (f+) = y_147_A330 (n-)
149 | A330 | LGAV | 21:15:00 | y_149_A330 (n+) = y_148_A330 (n-) + f_AR1866_A330 (f-)
376 | A330 | ZBAA | 14:20:00 | y_376_A330 (n+) + f_AR1161_A330 (f+) = y_377_A330 (n-)
377 | A330 | ZBAA | 11:25:00 | y_377_A330 (n+) = y_376_A330 (n-) + f_AR1160_A330 (f-)
58 | A330 | EFHK | 21:35:00 | y_58_A330 (n+) + f_AR1790_A330 (f+) = y_61_A330 (n-)
59 | A330 | EFHK | 14:52:00 | y_59_A330 (n+) = y_58_A330 (n-) + f_AR1755_A330 (f-)
60 | A330 | EFHK | 15:15:00 | y_60_A330 (n+) + f_AR1756_A330 (f+) = y_59_A330 (n-)
61 | A330 | EFHK | 21:07:00 | y_61_A330 (n+) = y_60_A330 (n-) + f_AR1759_A330 (f-)
0 | A330 | BIKF | 19:05:00 | y_0_A330 (n+) + f_AR1893_A330 (f+) = y_11_A330 (n-)
1 | A330 | BIKF | 01:35:00 | y_1_A330 (n+) = y_0_A330 (n-) + f_AR1824_A

In [ ]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].x) + ' (n+)'
            val += y[nodes[k][n][i]['n+'][0], k].x
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].x) + ' (f+)'
                val += f[w, k].x
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].x)  + ' (n-)'
            val -= y[nodes[k][n][i]['n-'][0], k].x
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].x) + ' (f-)'
                val -= f[w, k].x
            s += ' | Total ' + str(val)
            print(s)

146 | A330 | LGAV | 21:35:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
147 | A330 | LGAV | 07:40:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
148 | A330 | LGAV | 08:00:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
149 | A330 | LGAV | 21:15:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
376 | A330 | ZBAA | 14:20:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
377 | A330 | ZBAA | 11:25:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
58 | A330 | EFHK | 21:35:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
59 | A330 | EFHK | 14:52:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
60 | A330 | EFHK | 15:15:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
61 | A330 | EFHK | 21:07:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
0 | A330 | BIKF | 19:05:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
1 | A330 | BIKF | 01:35:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
2 | A330 | BIKF | 02:00:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
3 | A330 | BIKF | 09:25:00 | 0.0 (n+) = 0.0 (n-)

# Base model

In [274]:
# Define the model
m = gp.Model('IFAM')

# Notation
# P = paths with info on O, D, Demand, Fare, Legs
# L = flights list with info on AC Type, ORG, DEST, Departure, Arrival, Overnight, Capacity, Cost
# P0 = initial_pairs with key-value pair in initial pairs with key p and value 999 in a set
# K = Types of aircraft
# G = nodes with info on Node ID, AC Type, Airport, Time, Departures, Arrivals, n+, n-
# N = airports list
# Q = Q_i = unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
# R = recapture_p = set of passenger itineraries with Recapture Rate from itinerary p


# Decision variables from FAM
# f[i,k] [RELAXED] binary 1 if flight arc i is assigned to aircraft type k, 0 otherwise
f = {}
# y_ak = [RELAXED] integer number of aircraft of type k on the ground arc a
y = {}

for i in L:
    for k in K:
        f[i, k] = m.addVar(vtype=GRB.CONTINUOUS, name='f_' + str(i) + '_' + str(k))

for k in K:
    for a in list(ground_arcs[(ground_arcs['AC Type'] == k)]['Arc ID']):
        y[a, k] = m.addVar(vtype=GRB.CONTINUOUS,lb=0, name='y_' + str(a) + '_' + str(k))

# Decision variables from PMF
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}

for p in P0:
    for r in P0[p]:
        t[p,r] = m.addVar(vtype=GRB.CONTINUOUS,lb=0,name='t_'+str(p)+'_'+str(r))
m.update()

# Objective function part from the FAM
of = gp.quicksum(flights[i][k]['Cost'] * f[i,k] for i in L for k in K)

# Objective function part from the PMF
of +=  gp.quicksum((P[p]['Fare'] - R[(p,r)] * P[r]['Fare']) * t[p,r] 
                  for p in P0 for r in P0[p])

# Define the objective function
m.setObjective(of, GRB.MINIMIZE)

# Define the constraints from the FAM
# Constraint 1 [FAM]: 
# Each flight is assigned to exactly one aircraft type
m.addConstrs((gp.quicksum(f[i,k] for k in K) == 1 for i in L), name='one_ac')

# Constraint 2 [FAM]: 
# The number of AC arriving (n+ and arrivals) = AC departing yn-, for each type at each node
# y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
for k in K:
    for n in N:
        for i in G[k][n]:
            n_plus = y[G[k][n][i]['n+'][0], k]
            n_minus = y[G[k][n][i]['n-'][0], k]
            departures = gp.quicksum(f[w,k] for w in G[k][n][i]['Departures'])
            arrivals = gp.quicksum(f[w,k] for w in G[k][n][i]['Departures'])
            m.addConstr((n_plus + departures == n_minus + arrivals),
                         name='balance_' + str(i) + '_' + str(k) + '_' + str(n))

# Constraint 3 [FAM]: 
# The number of overnight arcs + the number of overnight flights = the number of aircraft of each type 
# using overnight_arcs and overnight_flights
# sum(y_a,k) + sum(f_i,k) = number of aircraft of type k
for k in K:
    m.addConstr((gp.quicksum(y[a, k] for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID'])) + 
                 gp.quicksum(f[i, k] for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.'])) == 
                 aircraft[k]['Units']), name='overnight_' + str(k))

# Constraint 4 [MIXED]: 
# Aircraft capacity constraint
# sum seats_k * f_ik -sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(aircraft[k]['Seats'] * f[i,k] for k in K) +
              gp.quicksum(s_ip[i,p] * t[p,r] for p in P0 for r in P0[p]) - 
              gp.quicksum(s_ip[i,p] * R[(r,p)] * t[p,r] for p in P0 for r in P0[p]) >= 
              Q[i] for i in L), name='π')

# Constraint 5 [PMF]: sum t_pr <= Dp for all p
for p in P0:
    m.addConstr((
        gp.quicksum(t[p,r] for r in P0[p]) <= 
        paths[p]['Demand']), name='σ[' + str(p) + ']')

# Update the model
m.update()
# Optimize the model but dont print the output
m.setParam('OutputFlag', 0)
m.optimize()
print('Objective value: %g' % (m.objVal/1000000))

Objective value: 5.41162


## Dual variables 

In [ ]:
# Print non-null dual variables

print('Dual variables:')
for c in m.getConstrs():
    if c.Pi != 0 and (c.constrName[0] == 'π' or c.constrName[0] == 'σ'):
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'π':
        flight_num_pi = c.ConstrName[2:-1]
        pi_dual[flight_num_pi] = c.Pi

sigma_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'σ':
        path_num_sigma = int(c.ConstrName[2:-1])
        sigma_dual[path_num_sigma] = c.Pi

Dual variables:
π[AR1133] = 57
π[AR1240] = 241
π[AR1241] = 288.235
π[AR1248] = 173
π[AR1249] = 47
π[AR1252] = 100
π[AR1253] = 51
π[AR1256] = 108
π[AR1260] = 23
π[AR1261] = 47
π[AR1263] = 49
π[AR1293] = 54
π[AR1303] = 262
π[AR1408] = 25
π[AR1410] = 1
π[AR1446] = 201
π[AR1447] = 103
π[AR1451] = 93
π[AR1458] = 6
π[AR1461] = 70
π[AR1462] = 101
π[AR1470] = 107
π[AR1471] = 94
π[AR1472] = 308
π[AR1473] = 105
π[AR1474] = 98
π[AR1475] = 109
π[AR1476] = 94
π[AR1477] = 104
π[AR1478] = 281
π[AR1657] = 3.07906
π[AR1759] = 1
π[AR1833] = 48
π[AR1836] = 103
σ[9] = -142
σ[19] = -145
σ[37] = -136
σ[47] = -2
σ[48] = -121
σ[58] = -95
σ[65] = -103
σ[69] = -17
σ[71] = -11
σ[72] = -12
σ[80] = -110
σ[94] = -7
σ[98] = -4
σ[106] = -9
σ[111] = -3
σ[112] = -103
σ[114] = -101
σ[117] = -3
σ[118] = -3
σ[119] = -3
σ[123] = -203
σ[124] = -215
σ[125] = -302
σ[126] = -216
σ[127] = -216
σ[128] = -203
σ[129] = -204
σ[130] = -204
σ[134] = -5
σ[147] = -11
σ[161] = -3
σ[172] = -2
σ[174] = -2
σ[194] = -7
σ[201] = -8
σ[202] = 

# NON RELAXED MODEL (FOR TESTING)

In [16]:
# Notation

L = flights_list
P = paths
K = ac_list
G = nodes
N = airports
Q = Q_i
R = recapture_p

In [19]:
R_test = {}
# if a recapture rate exists for p,r then keep it else set it to 0
for p in P:
    for r in P:
        if (p,r) in R.keys():
            R_test[p,r] = R[(p,r)]['Recapture Rate']
        else:
            R_test[p,r] = 0

In [26]:
# Define the model
m = gp.Model('IFAM')

# Notation
# P = paths with info on O, D, Demand, Fare, Legs
# L = flights_list
# P = initial_pairs
# K = ac_list
# G = nodes
# N = airports
# Q = Q_i
# R = recapture_p


# Decision variables from FAM
# f[i,k] [RELAXED] binary 1 if flight arc i is assigned to aircraft type k, 0 otherwise
f = {}
# y_ak = [RELAXED] integer number of aircraft of type k on the ground arc a
y = {}

for i in L:
    for k in K:
        f[i, k] = m.addVar(vtype=GRB.BINARY, name='f_' + str(i) + '_' + str(k))

for k in K:
    for a in list(ground_arcs[(ground_arcs['AC Type'] == k)]['Arc ID']):
        y[a, k] = m.addVar(vtype=GRB.INTEGER,lb=0, name='y_' + str(a) + '_' + str(k))

# Decision variables from PMF
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}

for p in P:
    for r in P:
        t[p,r] = m.addVar(vtype=GRB.CONTINUOUS,lb=0,name='t_'+str(p)+'_'+str(r))
m.update()

# Objective function part from the FAM
of = gp.quicksum(flights[i][k]['Cost'] * f[i,k]
                  for i in L for k in K)

# Objective function part from the PMF
of +=  gp.quicksum((P[p]['Fare'] - R_test[(p,r)] * P[r]['Fare']) * t[p,r] 
                  for p in P for r in P)

# Define the objective function
m.setObjective(of, GRB.MINIMIZE)

# Define the constraints from the FAM
# Constraint 1 [FAM]: 
# Each flight is assigned to exactly one aircraft type
m.addConstrs((gp.quicksum(f[i,k] for k in K) == 1 for i in L), name='one_ac')

# Constraint 2 [FAM]: 
# The number of AC arriving (n+ and arrivals) = AC departing yn-, for each type at each node
# y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
for k in K:
    for n in N:
        for i in G[k][n]:
            n_plus = y[G[k][n][i]['n+'][0], k]
            n_minus = y[G[k][n][i]['n-'][0], k]
            departures = gp.quicksum(f[w,k] for w in G[k][n][i]['Departures'])
            arrivals = gp.quicksum(f[w,k] for w in G[k][n][i]['Departures'])
            m.addConstr((n_plus + departures == n_minus + arrivals),
                         name='balance_' + str(i) + '_' + str(k) + '_' + str(n))

# Constraint 3 [FAM]: 
# The number of overnight arcs + the number of overnight flights = the number of aircraft of each type 
# using overnight_arcs and overnight_flights
# sum(y_a,k) + sum(f_i,k) = number of aircraft of type k
for k in K:
    m.addConstr((gp.quicksum(y[a, k] for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID'])) + 
                 gp.quicksum(f[i, k] for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.'])) == 
                 aircraft[k]['Units']), name='overnight_' + str(k))

# Constraint 4 [MIXED]: 
# Aircraft capacity constraint
# sum seats_k * f_ik -sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(aircraft[k]['Seats'] * f[i,k] for k in K) +
              gp.quicksum(s_ip[i,p] * t[p,r] for p in P for r in P) - 
              gp.quicksum(s_ip[i,p] * R_test[(r,p)] * t[r,p] for p in P for r in P) >= 
              Q[i] for i in L), name='π')

# Constraint 5 [PMF]: sum t_pr <= Dp for all p
for p in P:
    m.addConstr((
        gp.quicksum(t[p,r] for r in P) <= 
        paths[p]['Demand']), name='σ[' + str(p) + ']')

# Update the model
m.update()
# Optimize the model but dont print the output
m.setParam('OutputFlag', 1)
m.optimize()
print('Objective value: %g' % (m.objVal/1000000))

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2466 rows, 303730 columns and 795117 nonzeros
Model fingerprint: 0x9333264e
Variable types: 301401 continuous, 2329 integer (832 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [3e+01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 1.280083e+07
Presolve removed 2311 rows and 303257 columns
Presolve time: 0.44s
Presolved: 155 rows, 473 columns, 920 nonzeros
Found heuristic solution: objective 5434116.7452
Variable types: 372 continuous, 101 integer (101 binary)

Root relaxation: objective 5.403861e+06, 97 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

In [28]:
for w in ac_list:
    print('Flights for AC Type ' + w)
    for i in flights_list:
        for k in ac_list:
            if f[i,k].x > 0.1 and k == w:
                print(i," | ", round(f[i,k].x,ndigits=0)," | ", ('Cost = %0.3f' % (flights[i][k]['Cost'] * f[i,k].x / 1000)))
    print('\n')

# Count the non null values of f
total_flights = 0
for v in m.getVars():
    if v.x != 0 and v.varName[0] == 'f':
        total_flights += v.x
print('Total flights: ' + str(total_flights))
print(sum(f[i,k].x for i in flights for k in ac_list))


Flights for AC Type A330
AR1133  |  1.0  |  Cost = 20.187
AR1240  |  1.0  |  Cost = 13.769
AR1241  |  1.0  |  Cost = 13.971
AR1248  |  1.0  |  Cost = 13.769
AR1253  |  1.0  |  Cost = 14.439
AR1256  |  1.0  |  Cost = 13.971
AR1293  |  1.0  |  Cost = 14.527
AR1302  |  1.0  |  Cost = 19.349
AR1303  |  1.0  |  Cost = 18.788
AR1305  |  1.0  |  Cost = 18.313
AR1322  |  1.0  |  Cost = 18.402
AR1376  |  1.0  |  Cost = 17.352
AR1377  |  1.0  |  Cost = 17.404
AR1446  |  1.0  |  Cost = 12.578
AR1447  |  1.0  |  Cost = 12.287
AR1462  |  1.0  |  Cost = 12.578
AR1470  |  1.0  |  Cost = 12.287
AR1472  |  1.0  |  Cost = 12.133
AR1473  |  1.0  |  Cost = 12.578
AR1474  |  1.0  |  Cost = 12.287
AR1475  |  1.0  |  Cost = 12.578
AR1476  |  1.0  |  Cost = 12.287
AR1477  |  1.0  |  Cost = 12.578
AR1478  |  1.0  |  Cost = 12.287
AR1833  |  1.0  |  Cost = 13.098
AR1836  |  1.0  |  Cost = 13.265
AR1920  |  1.0  |  Cost = 19.539
AR1921  |  1.0  |  Cost = 19.539


Flights for AC Type A340
AR1304  |  1.0  |  Cost 

In [ ]:
        # for i in L:
        #     for k in K:
        #         if f[i,k].x > 0:
        #             print(i, aircraft[k]['Seats'], f[i,k].varName)
        #     print('End of sum 1')
        #     for p in  current_pairs:
        #         for r in  current_pairs[p]:
        #             if s_ip[i,p] == 1:
        #                 print(i, s_ip[i,p], t[p,r].VarName)
        #     print('End of sum 2')
        #     for p in  current_pairs:
        #         for r in  current_pairs[p]:
        #             if s_ip[i,r] == 1:
        #                 print(i, s_ip[i,r], R[(p,r)]['Recapture Rate'], t[p,r].VarName)
        #     print('End of sum 3')
        #     print(Q_i[i])

        
        # print('\nDual variables:')
        # for c in m_n.getConstrs():
        #     if c.Pi != 0 and (c.constrName[0] == 'π' or c.constrName[0] == 'σ'):
        #         print('%s = %g' % (c.ConstrName, c.Pi))